# Initializing parameters and Loading data

## Imports

In [6]:
# imports
%cd /content/drive/My\ Drive/Thesis/
import matplotlib.pyplot as plt
from NNClasses import *
from NNDataLoader import *
import os
from torch.utils.data import DataLoader
import torch.optim as optim
from time import time
!pip install skorch
from skorch import NeuralNetBinaryClassifier, NeuralNetClassifier
from skorch.dataset import CVSplit
from skorch.callbacks import EarlyStopping, EpochScoring
import skorch.history
%load_ext autoreload
%autoreload 1
%aimport NNClasses
%aimport NNDataLoader

/content/drive/My Drive/Thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parameters

In [7]:
# training params
epochs = 5 # 250
criterion = nn.BCELoss # TODO CHANGE THIS TO NOT BINARY CROSS ENTROPY LOSS BUT NORMAL CROSS ENTROPY LOSS, THANK YOU AND COME AGAIN
criterion = nn.BCEWithLogitsLoss
optimizer = optim.SGD
learning_rate = 1e-3
batch_size = 10
cv_splt_size = .10
early_stop_patience = 50
Tc0 = .9
L = 16

max_data_Size = 50 # 0 = no limit
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Drive load data

In [8]:
# general data path
root = os.path.join("thesis", "data", "configs")

# load train, todo: and test set
t1 = time()
print(f"root folder: {root}"                    )
train_data = CustomDataset(root+"Train", L, Tc0, data_Size=max_data_Size, type_="binary")

t2 = time()
#test_data = CustomDataset(root+"Test", L, Tc0, data_Size=max_data_Size)
print(f"Time for loading training data: {t2-t1:.2f}s")
print(f"Train set size: {len(train_data)} elements")
#print(f"Test set size: {len(test_data)} elements")

root folder: thesis/data/configs
Time for loading training data: 18.61s
Train set size: 50 elements


### Vortices or spin angles

In [9]:
# train on spins or vortices?
# uncomment next line to train on vortices
#train_data.spin_to_vortex()
print(f"training on: {train_data.state}")

training on: spin angles


# Models and training

In [20]:
def MODEL(module, L_=L):
  return  NeuralNetBinaryClassifier(
      module=module(L=L_),
      max_epochs=epochs,
      criterion=criterion,
      optimizer=optimizer,
      lr = learning_rate,
      batch_size = batch_size,
      iterator_train__shuffle=False, # shuffle train data each epoch
      train_split = CVSplit(cv_splt_size), # use sklearn cv split instead
      callbacks=[
          ('early_stopping', EarlyStopping(patience=early_stop_patience)),
          ('epoch_scoring', EpochScoring("accuracy", lower_is_better=False, 
                      on_train=True, name="train_acc"))
      ],
      device=device,
  )

# initialize models
netFFNN = MODEL(FFNN)
netCNN = MODEL(CNN)
netCustom = MODEL(Custom)

# don't train custom NN on vortices, it makes no sense
if train_data.state == "vortices":
  nets = netFFNN, netCNN
else:
  nets = netFFNN, netCNN, netCustom

nets = netFFNN, netCNN
modelnames = ["FFNN", "CNN", "Custom"]

print("y_true/target shape", train_data.labels[0:10].shape)
# train the models
histories = []
for i,net in enumerate(nets):
    print("currently training",modelnames[i], "model")
    
    net.initialize()
    print(net.predict(train_data))
    print(net.predict_proba(train_data))
    print(train_data.labels.shape)
    net.fit(train_data, y=None)
    break
    histories.append(net.history)

y_true/target shape torch.Size([10, 1])
currently training FFNN model
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[0.3892349  0.6107651 ]
 [0.3905633  0.6094367 ]
 [0.3893103  0.6106897 ]
 [0.39507246 0.60492754]
 [0.38629025 0.61370975]
 [0.3871312  0.6128688 ]
 [0.38527298 0.614727  ]
 [0.40185022 0.5981498 ]
 [0.3953765  0.6046235 ]
 [0.38100755 0.61899245]
 [0.3857646  0.6142354 ]
 [0.38816196 0.61183804]
 [0.40069067 0.5993093 ]
 [0.3910933  0.6089067 ]
 [0.40074933 0.5992507 ]
 [0.39522308 0.6047769 ]
 [0.38969094 0.61030906]
 [0.39667052 0.6033295 ]
 [0.39207    0.60793   ]
 [0.3957497  0.6042503 ]
 [0.38249236 0.61750764]
 [0.39119476 0.60880524]
 [0.3761829  0.6238171 ]
 [0.38851368 0.6114863 ]
 [0.38882697 0.61117303]
 [0.38712496 0.61287504]
 [0.39662534 0.60337466]
 [0.38992494 0.61007506]
 [0.37940943 0.62059057]
 [0.3935058  0.6064942 ]
 [0.39018285 0.60981715]
 [0.39495635 0.60504365]
 [0.38974428 0.6102557 ]
 [

ValueError: ignored

## Plot learning curves

In [ ]:
def plot_train_valid_score(ax, history, modelname, labels=("train_loss", "valid_loss")):
    data = zip(*history[:, labels])
    epochs = len(history)
    for i, xdata in enumerate(data):
        # check if earlystopping activated, then last epoch won't have all values
        if any([lastvalue == skorch.history._none for lastvalue in history[-1, labels]]):
            xrange=range(epochs-1)
        else:
            xrange=range(epochs)
        
        ax.plot(xrange, xdata, label=labels[i])
        
        ax.set_title(modelname)
    ax.set_xlabel("Epochs trained")
    ax.set_ylabel("")
    # ax.set_ylim(0,1)
    ax.set_aspect("auto")
    ax.legend()

In [ ]:
fig = plt.figure(figsize=(15,7), dpi=100)
axs = [fig.add_subplot(1,len(histories),i+1) for i in range(len(histories))]
labels=("train_loss", "valid_loss", "train_acc", "valid_acc")
for i, history in enumerate(histories):
    plot_train_valid_score(axs[i], history, modelnames[i], labels)


plt.show()

In [ ]:
# # Save plot
# filename = f"Training_epochs={epochs}_L={L}.png"
# filename = os.path.join("Images", filename)
# fig.savefig(filename)

# Grid search

In [ ]:
# function used to plot the grid search scores
def plot_search_results(grid):
    """
    Params: 
        grid: A trained GridSearchCV object.
    """
    ## Results from grid search
    results = grid.cv_results_
    means_test = results['mean_test_score']
    stds_test = results['std_test_score']

    ## Getting indexes of values per hyper-parameter
    masks=[]
    masks_names= list(grid.best_params_.keys())
    for p_k, p_v in grid.best_params_.items():
        masks.append(list(results['param_'+p_k].data==p_v))

    params=grid.param_grid

    ## Ploting results
    fig, ax = plt.subplots(1,len(params),sharex='none', sharey='all',figsize=(20,5))
    fig.suptitle('Score per parameter')
    fig.text(0.04, 0.5, 'MEAN SCORE', va='center', rotation='vertical')
    pram_preformace_in_best = {}
    for i, p in enumerate(masks_names):
        m = np.stack(masks[:i] + masks[i+1:])
        pram_preformace_in_best
        best_parms_mask = m.all(axis=0)
        best_index = np.where(best_parms_mask)[0]
        x = np.array(params[p])
        y_1 = np.array(means_test[best_index])
        e_1 = np.array(stds_test[best_index])
        ax[i].errorbar(x, y_1, e_1, linestyle='--', marker='o', label='test')
        ax[i].set_xlabel(p.upper())

    plt.legend()
    plt.show()

In [ ]:
# gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, log_loss

# params
params = {
    'lr': [.1,.05],
    'iterator_train__batch_size': [2,5]
}

# which net to gridsearch on
net = netFFNN

# perform the gs
gs = GridSearchCV(net, params, refit=True, cv=10, scoring=make_scorer(log_loss))
X, y = train_data.configs.cpu().numpy(), train_data.labels.cpu().numpy()
X = np.expand_dims(X, 1)
gs.fit(X, y)

In [ ]:
# plot the results
plot_search_results(gs)